# Feature Processing

En este notebook obtenemos los features de interés a partir de las matrices de mediciones de los sujetos.

### Imports

In [1]:
%pylab
%matplotlib inline

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [45]:
from __future__ import print_function, division, unicode_literals
import glob
import os.path
import pickle
import scipy.io as sio
import scipy.signal as scs
import pandas as pd

## Definiciones globales

In [19]:
pickle_dir = 'pickles'

In [23]:
if not os.path.isdir(pickle_dir):
    os.mkdir(pickle_dir)

In [34]:
dataDir = 'dataset'
fs = 250

## Identificación de sujetos

In [75]:
SUBJECT_FILES = 'subjects_files.pkl'
subjects_files_path = os.path.join(pickle_dir, SUBJECT_FILES)

if not os.path.isfile(subjects_files_path):
    subjects = glob.glob('dataset/*.mat')
    matrix_files = sorted(subjects)  # dataset/P01.mat ...
    matrix_names = [os.path.basename(n).replace('.mat', '') for n in matrix_files]  # P01.mat ...
    
    with open(os.path.join(pickle_dir, SUBJECT_FILES), 'wb') as f:
        pickle.dump(matrix_files, f, protocol=2)
else:
    with open(os.path.join(pickle_dir, SUBJECT_FILES), 'rb') as f:
        matrix_files = pickle.load(f)  # dataset/P01.mat ...
        matrix_names = [os.path.basename(n).replace('.mat', '') for n in matrix_files]  # P01.mat ...

## Potencia en bandas

In [90]:
# Nombres de archivos

BAND_POWER_MEAN = 'band_power_mean.pkl'
BAND_POWER_STD = 'band_power_std.pkl'
BAND_POWER_NORM_MEAN = 'band_power_norm_mean.pkl'
BAND_POWER_NORM_STD = 'band_power_norm_std.pkl'
band_power_mean_file = os.path.join(pickle_dir, BAND_POWER_MEAN)
band_power_std_file = os.path.join(pickle_dir, BAND_POWER_STD)
band_power_norm_mean_file = os.path.join(pickle_dir, BAND_POWER_NORM_MEAN)
band_power_norm_std_file = os.path.join(pickle_dir, BAND_POWER_NORM_STD)

In [88]:
# Inicialización variables
band_names = ['alpha', 'beta', 'gamma', 'delta', 'theta']

band_means = pd.DataFrame(index=matrix_names, columns=band_names)
band_norm_means= pd.DataFrame(index=matrix_names, columns=band_names)
band_std = pd.DataFrame(index=matrix_names, columns=band_names)
band_norm_std= pd.DataFrame(index=matrix_names, columns=band_names)

In [ ]:
# Calculo de información

for idx in range(len(matrix_names)):
    file = matrix_files[idx]
    print(file)
    subject_name = matrix_names[idx]
    data = sio.loadmat(file)
    data_array = data['data']

    # potencia: epoch x electrodo x freq
    freq, Pxx_trials_subjects = scs.welch(data_array[:,:,:], fs=fs)

    # potencia: epoch x freq 
    Pxx_epoch_subjects = np.mean(Pxx_trials_subjects, axis=1)

    # potencia (de banda): epoch
    Pxx_epoch_delta_subjects = np.sum(Pxx_epoch_subjects[:, freq < 4], axis=1)
    Pxx_epoch_theta_subjects = np.sum(Pxx_epoch_subjects[:, (freq >= 4)*(freq < 8)], axis=1)
    Pxx_epoch_beta_subjects = np.sum(Pxx_epoch_subjects[:, (freq >= 13)*(freq < 30)], axis=1)
    Pxx_epoch_gamma_subjects = np.sum(Pxx_epoch_subjects[:, freq >= 30], axis=1)
    Pxx_epoch_alpha_subjects = np.sum(Pxx_epoch_subjects[:, freq >= 30], axis=1)
    
    band_means['delta'][subject_name] = Pxx_epoch_delta_subjects.mean()
    band_means['theta'][subject_name] = Pxx_epoch_theta_subjects.mean()
    band_means['beta'][subject_name] = Pxx_epoch_beta_subjects.mean()
    band_means['gamma'][subject_name] = Pxx_epoch_gamma_subjects.mean()
    band_means['alpha'][subject_name] = Pxx_epoch_alpha_subjects.mean()
    
    band_std['delta'][subject_name] = Pxx_epoch_delta_subjects.std()
    band_std['theta'][subject_name] = Pxx_epoch_theta_subjects.std()
    band_std['beta'][subject_name] = Pxx_epoch_beta_subjects.std()
    band_std['gamma'][subject_name] = Pxx_epoch_gamma_subjects.std()
    band_std['alpha'][subject_name] = Pxx_epoch_alpha_subjects.std()
    
    band_sum = np.array([
        Pxx_epoch_delta_subjects,
        Pxx_epoch_theta_subjects,
        Pxx_epoch_beta_subjects,
        Pxx_epoch_gamma_subjects,
        Pxx_epoch_alpha_subjects
    ]).sum(axis=0)
    
    band_means['delta'][subject_name] = (Pxx_epoch_delta_subjects / band_sum).mean()
    band_means['theta'][subject_name] = (Pxx_epoch_theta_subjects / band_sum).mean()
    band_means['beta'][subject_name] = (Pxx_epoch_beta_subjects / band_sum).mean()
    band_means['gamma'][subject_name] = (Pxx_epoch_gamma_subjects / band_sum).mean()
    band_means['alpha'][subject_name] = (Pxx_epoch_alpha_subjects / band_sum).mean()
    
    band_std['delta'][subject_name] = (Pxx_epoch_delta_subjects / band_sum).std()
    band_std['theta'][subject_name] = (Pxx_epoch_theta_subjects / band_sum).std()
    band_std['beta'][subject_name] = (Pxx_epoch_beta_subjects / band_sum).std()
    band_std['gamma'][subject_name] = (Pxx_epoch_gamma_subjects / band_sum).std()
    band_std['alpha'][subject_name] = (Pxx_epoch_alpha_subjects / band_sum).std()

In [87]:
# Grabación
with open(band_power_mean_file, 'wb') as f:
    pickle.dump(band_means, f, protocol=2)

with open(band_power_std_file, 'wb') as f:
    pickle.dump(band_std, f, protocol=2)

with open(band_power_norm_mean_file, 'wb') as f:
    pickle.dump(band_means, f, protocol=2)

with open(band_power_norm_std_file, 'wb') as f:
    pickle.dump(band_std, f, protocol=2)